# Configuración inicial de PySpark

In [ ]:
!pip install pyspark

# Creación de una nueva sesión spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark import SparkContext

spark = SparkSession.builder.master('local[*]').appName('pyspark-test').getOrCreate()
sc = SparkContext.getOrCreate()

# Lectura de datos California Housing

En estos pasos leemos un csv en un dataframe y mostramos información básica como varias filas del dataframe y el schema inferido al cargar dichos datos.

In [ ]:
df = spark.read.csv('sample_data/california_housing_train.csv', inferSchema = True, header = True)

In [ ]:
df.show(5)

In [ ]:
df.printSchema()

# Uso de Select

In [ ]:
df.select('population', 'median_income').show(2)

# Uso de Filter

In [ ]:
# Uso con formato cadena
df.filter('population > 1000').show()

In [ ]:
# Uso mediante atributos del dataframe


In [ ]:
# Uso de filtros compuestos con formato cadena


In [ ]:
# Uso de filtros compuestos mediante atributos del dataframe


# Añadir columnas

In [ ]:
df.withColumn('latitude_longitude', df.latitude * df.longitude).show()

In [ ]:
df_ll = df.withColumnRenamed('longitude', 'long').withColumnRenamed('latitude', 'lat')

# Funciones SQL
En esta sección aplicamos varias funciones a las columnas del dataframe utilizando agg para crear una agregación de columnas.

In [ ]:
from pyspark.sql.functions import sum, avg

df.agg(sum('median_house_value'), avg('median_house_value')).show()

+-----------------------+-----------------------+
|sum(median_house_value)|avg(median_house_value)|
+-----------------------+-----------------------+
|           3.52411551E9|     207300.91235294117|
+-----------------------+-----------------------+



In [ ]:
from pyspark.sql.functions import count

df.agg(count('housing_median_age')).show()

+-------------------------+
|count(housing_median_age)|
+-------------------------+
|                    17000|
+-------------------------+



In [ ]:
from pyspark.sql.functions import countDistinct

df.agg(countDistinct('housing_median_age')).show()

+-------------------------+
|count(housing_median_age)|
+-------------------------+
|                       52|
+-------------------------+



In [ ]:
from pyspark.sql.functions import sum, when

df.agg(sum(when(df['housing_median_age'] > 15, 1).otherwise(0))).show()

+----------------------------------------------------------+
|sum(CASE WHEN (housing_median_age > 15) THEN 1 ELSE 0 END)|
+----------------------------------------------------------+
|                                                     14275|
+----------------------------------------------------------+



In [ ]:
df.filter(df.latitude > 40).agg(sum(when(df['population'] > 1000, df.population).otherwise(0))).show()

+-------------------------------------------------------------+
|sum(CASE WHEN (population > 1000) THEN population ELSE 0 END)|
+-------------------------------------------------------------+
|                                                     227602.0|
+-------------------------------------------------------------+



In [ ]:
df.filter((df.latitude > 40) & (df.population > 1000)).agg(sum(df.population).alias('sum_population')).show()

+--------------+
|sum_population|
+--------------+
|      227602.0|
+--------------+



# Creando vistas para trabajar con SQL sobre los datos de un dataframe


In [ ]:
df.sort('housing_median_age').createOrReplaceTempView('hma_ordered')

In [ ]:
spark.sql('select * from hma_ordered').show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -120.93|   37.65|               1.0|     2254.0|         328.0|     402.0|     112.0|         4.25|          189200.0|
|   -122.0|   38.23|               1.0|     2062.0|         343.0|     872.0|     268.0|       5.2636|          191300.0|
|  -121.35|   38.46|               2.0|     6992.0|        1132.0|    2816.0|     984.0|       4.3879|          144400.0|
|  -117.62|   33.64|               2.0|     7826.0|         893.0|    2985.0|     790.0|      10.1531|          484100.0|
|  -117.61|   33.63|               2.0|     4678.0|         817.0|    1970.0|     712.0|       6.1078|          219000.0|
|  -117.65|   33.58|    

In [ ]:
from pyspark.sql.functions import col
df.sort(col('households').desc()).createOrReplaceTempView('hh_order_desc')

In [ ]:
spark.sql('select count(longitude) from hh_order_desc').show()

+----------------+
|count(longitude)|
+----------------+
|           17000|
+----------------+



In [ ]:
spark.sql('select longitude from hh_order_desc where total_bedrooms > 1000').count()

1576

In [ ]:
spark.sql('select count(longitude) from hh_order_desc where total_bedrooms > 1000').show()

+----------------+
|count(longitude)|
+----------------+
|            1576|
+----------------+



In [ ]:
df.select('longitude', 'latitude').filter(df.longitude > 33).agg(sum(df.latitude), avg(df.longitude)).show()

+-------------+--------------+
|sum(latitude)|avg(longitude)|
+-------------+--------------+
|         null|          null|
+-------------+--------------+



In [ ]:
df_bmi = df.withColumn('big_median_income', df.median_income > 5.2636).filter(col('big_median_income'))

In [ ]:
df_bmi.agg(sum(when(df_bmi.big_median_income, 1).otherwise(0))).show()

+--------------------------------------------------+
|sum(CASE WHEN big_median_income THEN 1 ELSE 0 END)|
+--------------------------------------------------+
|                                              3105|
+--------------------------------------------------+



In [ ]:
df.filter((df.latitude > 33) & (df.latitude <= 34)).agg(sum(df.latitude).alias('sum_latitude')).show()

+-----------------+
|     sum_latitude|
+-----------------+
|139155.3799999997|
+-----------------+



In [ ]:
df.filter('latitude > 33 and latitude <= 34').agg(sum(df.latitude), avg(df.longitude)).show()

+-----------------+-----------------+
|    sum(latitude)|   avg(longitude)|
+-----------------+-----------------+
|139155.3799999997|-117.875031553398|
+-----------------+-----------------+



In [ ]:
from pyspark.sql.functions import avg, sum

df.agg(avg(df.latitude), sum(df.latitude)).show()

+----------------+-----------------+
|   avg(latitude)|    sum(latitude)|
+----------------+-----------------+
|35.6252247058827|605628.8200000059|
+----------------+-----------------+



In [ ]:
df_ratings = spark.read.csv('/content/drive/MyDrive/CursoBigData/BigDataAplicado/movies/ratings_small.csv', inferSchema = True, header = True)

In [ ]:
df_ratings.show(10)

+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|     1|     31|   2.5|1260759144|
|     1|   1029|   3.0|1260759179|
|     1|   1061|   3.0|1260759182|
|     1|   1129|   2.0|1260759185|
|     1|   1172|   4.0|1260759205|
|     1|   1263|   2.0|1260759151|
|     1|   1287|   2.0|1260759187|
|     1|   1293|   2.0|1260759148|
|     1|   1339|   3.5|1260759125|
|     1|   1343|   2.0|1260759131|
+------+-------+------+----------+
only showing top 10 rows



In [ ]:
df_metadata = spark.read.csv('/content/drive/MyDrive/CursoBigData/BigDataAplicado/movies/movies_metadata.csv', inferSchema = True, header = True)

In [ ]:
df_metadata.show(2)

+-----+---------------------+--------+--------------------+--------------------+----+---------+-----------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+---------+-----+------------+----------+
|adult|belongs_to_collection|  budget|              genres|            homepage|  id|  imdb_id|original_language|original_title|            overview|popularity|         poster_path|production_companies|production_countries|release_date|  revenue|runtime|    spoken_languages|  status|             tagline|    title|video|vote_average|vote_count|
+-----+---------------------+--------+--------------------+--------------------+----+---------+-----------------+--------------+--------------------+----------+--------------------+--------------------+--------------------+------------+---------+-------+--------------------+--------+--------------------+---

In [ ]:
spark.sql('SELECT * FROM hh_order_desc').show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -121.79|   36.64|              11.0|    32627.0|        6445.0|   28566.0|    6082.0|       2.3087|          118800.0|
|  -117.74|   33.89|               4.0|    37937.0|        5471.0|   16122.0|    5189.0|       7.4947|          366300.0|
|  -117.78|   34.03|               8.0|    32054.0|        5290.0|   15507.0|    5050.0|       6.0191|          253900.0|
|  -117.42|   33.35|              14.0|    25135.0|        4819.0|   35682.0|    4769.0|       2.5729|          134400.0|
|  -117.34|   33.98|              10.0|    17286.0|        4952.0|    9851.0|    4616.0|       1.7579|          103400.0|
|  -118.46|    34.4|    